In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.13771462440490723
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 62.37169931063647
200000 104.13010068281613
300000 105.5573894016689
400000 104.54044452931097
500000 101.63858644403366
600000 101.78234897539849
700000 106.95047329039022
800000 101.89816652874936
900000 100.78257560022111
1000000 100.42762883776429
1100000 100.86060563110345
1200000 100.16318170877338
1300000 81.63470724482198
1400000 40.17539483693253
1500000 99.83246696379626
1600000 99.71648890802669
1700000 99.68714623692927
1800000 98.57581525782386
1900000 99.48184468296704
2000000 103.18178300484449
2100000 101.15752523075365
2200000 98.7142394660706
2300000 98.82804488400737
2400000 100.15778802251987
2500000 100.35275914669536
2600000 96.4391437521379
2700000 24.404637699417886
2800000 97.06727175205044
2900000 99.10614531013502
3000000 98.21

29400000 111.66326313619199
29500000 117.48451035837151
29600000 116.70483243056775
29700000 111.58424427288845
29800000 115.49235792934095
29900000 115.0810931215832
30000000 105.25399896304944
30100000 112.53860698169692
30200000 90.55336757914556
30300000 43.36415576001163
30400000 101.60943036927273
30500000 113.3823975447866
30600000 106.94736566894184
30700000 126.15496383865036
30800000 113.39337161269205
30900000 105.33470306356028
31000000 122.99584220832408
31100000 118.32600311856342
31200000 107.3139034849512
31300000 115.04961007571677
31400000 44.628567903655714
31500000 119.34460449709
31600000 117.17483265746259
31700000 112.7032786826132
31800000 54.39347578762453
31900000 92.9747782619735
32000000 112.62934398283676
32100000 105.34350387609913
32200000 111.64573690032911
32300000 106.47161294233642
32400000 103.69381142312098
32500000 111.08989671304803
32600000 100.85462944651772
32700000 107.57702214729838
32800000 98.4722707241698
32900000 31.47211811340143
3300000

59200000 23.073655651680873
59300000 96.29950011671471
59400000 93.66988737247601
59500000 94.89438728480266
59600000 95.89819837475105
59700000 98.07439811735502
59800000 98.37559582283657
59900000 100.14882255726144
60000000 99.67858253007203
60100000 43.766189351109524
60200000 103.88328349031725
60300000 102.90452962803835
60400000 103.57877371763814
60500000 104.82388531873077
60600000 110.48785982075155
60700000 106.49078081708554
60800000 106.82705169250863
60900000 65.09406210148938
61000000 76.6599830652291
61100000 107.1128748311891
61200000 101.17977303797352
61300000 102.22741518586481
61400000 109.81852793874765
61500000 97.15760208285204
61600000 95.85431848372198
61700000 94.9546203193281
61800000 33.94554352160378
61900000 88.05920677361588
62000000 97.32336223523168
62100000 96.79583499443967
62200000 103.45803965612515
62300000 98.9451984624121
62400000 99.5193352043894
62500000 104.06806840888868
62600000 104.99549055439502
62700000 35.85251491632743
62800000 111.735

89200000 106.53062947753081
89300000 111.33470995437409
89400000 110.08993790498702
89500000 107.50014107782881
89600000 101.48114355849782
89700000 101.79702990832553
89800000 103.33447287598375
89900000 98.87458498863919
90000000 76.05045829976123
90100000 55.3005506254712
90200000 91.61814973892724
90300000 89.4008280770793
90400000 89.71815202417642
90500000 92.8230522843072
90600000 91.30806591271939
90700000 39.34431725563008
90800000 77.23731254602151
90900000 92.68668487210859
91000000 90.51706936916916
91100000 90.68815294810435
91200000 91.23716538194545
91300000 90.93634413794425
91400000 33.98824906018073
91500000 100.92105438846866
91600000 99.35152800322436
91700000 101.16304909051452
91800000 100.28030105572131
91900000 102.96066824402801
92000000 110.8108633664443
92100000 107.25749967892388
92200000 105.47006849107161
92300000 101.89689750073802
92400000 99.43126827548922
92500000 99.43725979468026
92600000 99.48229012153253
92700000 29.93605470182679
92800000 89.04010

118900000 89.70124840753809
119000000 88.28845920427645
119100000 79.82792529516428
119200000 38.574271575311975
119300000 86.4414361958916
119400000 90.05182674293602
119500000 88.0521324410044
119600000 44.755872921691825
119700000 67.20380665568474
119800000 88.34162680022486
119900000 87.82338041649228
120000000 88.51020043941591
120100000 32.681906889625324
120200000 89.90194186896503
120300000 89.89002598010106
120400000 90.95116933189004
120500000 95.17494592443695
120600000 93.53561706316945
120700000 100.23222909164512
120800000 97.25451762457267
120900000 101.32102318937348
121000000 90.86720552324319
121100000 93.5624345714594
121200000 94.83882986756251
121300000 95.22029874645712
121400000 88.14321585051145
121500000 88.16688871126162
121600000 85.4465631844311
121700000 87.16286505552148
121800000 42.4785213132037
121900000 72.28695353278614
122000000 83.84065494279322
122100000 84.05560673240781
122200000 62.5462270034085
122300000 44.88542504481698
122400000 85.16166696

...

982900000 9.546934497119715
983000000 5.082896701746708
983100000 6.753156685443688
983200000 9.595768200298924
983300000 10.552103325605216
983400000 8.635199941591068
983500000 7.644077016053695
983600000 6.698334307769097
983700000 1.825094341289537
983800000 7.3127546814647655
983900000 7.516067874651911
984000000 8.177098246808196
984100000 6.909624741665435
984200000 6.632404357285718
984300000 3.8336109608651268
984400000 3.553242700840563
984500000 6.89690118911198
984600000 6.636620215819233
984700000 7.8748225034866834
984800000 6.809638753799271
984900000 6.134606747327499
985000000 3.2190623578666324
985100000 5.382325174863384
985200000 6.863010727511784
985300000 7.515847338575223
985400000 7.420578292411077
985500000 6.9010873219418425
985600000 4.878280540276891
985700000 2.979571714865243
985800000 6.799312211957873
985900000 7.116740064741541
986000000 7.804409556748834
986100000 6.888878021441173
986200000 6.006499902062309
986300000 3.8898020594127325
986400000 5.38

1011400000 3.9104883296998385
1011500000 3.421163531082813
1011600000 4.067214169968391
1011700000 3.73612955480963
1011800000 3.700845447688939
1011900000 1.848529055226493
1012000000 2.254229418859589
1012100000 3.4005451259184025
1012200000 3.6438381783981093
1012300000 4.003797598459911
1012400000 3.1169783638014943
1012500000 2.7334961405585845
1012600000 0.7900773342692752
1012700000 2.5314777851518366
1012800000 3.1855507810501025
1012900000 3.5135323967211303
1013000000 3.0306342811393727
1013100000 2.7045891460798046
1013200000 1.6349024306492883
1013300000 1.1794526054951973
1013400000 3.006287189649347
1013500000 2.784065178267348
1013600000 3.194353964380455
1013700000 2.5487882775224717
1013800000 2.5102484193803427
1013900000 0.8442855143210564
1014000000 1.9784816819566993
1014100000 2.444564293773637
1014200000 2.622857642336535
1014300000 2.5374962519639923
1014400000 2.190205328893695
1014500000 1.718862385044559
1014600000 0.8979787058279866
1014700000 2.538747019407

1037200000 1.4639394250689983
1037300000 1.672719776593115
1037400000 1.9512425541175114
1037500000 1.4883464687341887
1037600000 1.6611902119129067
1037700000 1.8126936751297376
1037800000 1.628928363636879
1037900000 1.9873877779974713
1038000000 1.6255584253560245
1038100000 1.5196490294583738
1038200000 1.8727067976850893
1038300000 1.692917986051038
1038400000 1.47948541688234
1038500000 1.7007285687293028
1038600000 1.8209842820128734
1038700000 1.4892303169979493
1038800000 2.2090077006084234
1038900000 1.9373136582472474
1039000000 1.5531071539562582
1039100000 1.5639559583865807
1039200000 2.0842817156820574
1039300000 1.6566766656509022
1039400000 1.3768800621521737
1039500000 2.338073151711748
1039600000 1.7684551999035936
1039700000 1.2845958587373407
1039800000 1.9313345438122147
1039900000 1.8290438741274118
1040000000 1.642674021185406
1040100000 2.0104131519672084
1040200000 2.2826028322554968
best so far: 0
type: [1, 3, 1, 2, 23] 138
cases of this type: 6716184
1040300

1058300000 0.8541278779369585
1058400000 0.7862277751637544
1058500000 1.1539416746189375
1058600000 0.9231617679190206
1058700000 0.9390936074931311
1058800000 0.985834168609321
1058900000 0.9265174313037337
1059000000 0.8978704563435879
1059100000 0.880203555144769
1059200000 1.0978592705485708
1059300000 0.7253630124019543
1059400000 0.830771309894991
1059500000 1.0835385927051995
1059600000 0.8368323882708166
1059700000 0.923122869896251
1059800000 1.020121866685674
1059900000 0.8754030157652425
1060000000 0.8108241463009357
1060100000 0.899026211929875
1060200000 1.04746077403399
1060300000 0.6787598035039802
1060400000 0.8735286895447473
1060500000 1.054223086815207
1060600000 0.8119825306015849
1060700000 0.703684584343593
1060800000 1.016900067728942
1060900000 0.8132894908164203
1061000000 0.661600524994313
1061100000 0.9528977340517556
1061200000 0.9256721604588787
1061300000 0.6291949957293981
1061400000 1.050351646653027
1061500000 0.9559535994631112
1061600000 0.6645623932

best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.1329504559418387
1084900000 0.20733833603533505
1085000000 0.18229897175683113
1085100000 0.16978037850011188
1085200000 0.16959992253371875
1085300000 0.16512033416345384
1085400000 0.16961708751933574
1085500000 0.16722339251193202
1085600000 0.16806977183377014
1085700000 0.17571257681217992
1085800000 0.10452329330824614
1085900000 0.05799678818471498
1086000000 0.05778546562007467
1086100000 0.05688985026707451
1086200000 0.0565541045779533
1086300000 0.08382481034453974
1086400000 0.18621601058694323
1086500000 0.15969981956211857
1086600000 0.15253889277409846
1086700000 0.1508454627604127
1086800000 0.1452850041615248
1086900000 0.14440189733106626
1087000000 0.14120097956122557
1087100000 0.14797630175547402
1087200000 0.15994294763553277
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.1348941669663648
best so far: 0
type: [2, 1, 23, 1, 3] 138
cases of this type